In [0]:
import numpy as np
import matplotlib.pyplot as plt
import imgaug as ia
import cv2 as cv

%matplotlib inline

np.random.seed(1)

In [1]:
!tar -xvf ../qr_codes.tar.gz
!rm -rf images/
!mv qr_codes/* ./
!rm -r qr_codes/

qr_codes/
qr_codes/fips_valid.csv
qr_codes/images/
qr_codes/images/5327274374.jpg
qr_codes/images/5014378360.jpg
qr_codes/images/2126492335.jpg
qr_codes/images/4872978369.jpg
qr_codes/images/4821931054.jpg
qr_codes/images/4946341553.jpg
qr_codes/images/2332977452.jpg
qr_codes/images/3210084213.jpg
qr_codes/images/4917222006.jpg
qr_codes/images/3359230395.jpg
qr_codes/images/4905458230.jpg
qr_codes/images/5236047029.jpg
qr_codes/images/5208855909.jpg
qr_codes/images/4340207281.jpg
qr_codes/images/3027815370.jpg
qr_codes/images/5326665675.jpg
qr_codes/images/4807729131.jpg
qr_codes/images/5117983633.jpg
qr_codes/images/2078613193.jpg
qr_codes/images/2237317748.jpg
qr_codes/images/3643877605.jpg
qr_codes/images/3303278929.jpg
qr_codes/images/4787848106.jpg
qr_codes/images/5121116199.jpg
qr_codes/images/4649988496.jpg
qr_codes/images/3594871243.jpg
qr_codes/images/3360044712.jpg
qr_codes/images/4994326352.jpg
qr_codes/images/3976044005.jpg
qr_codes/images/4941942874.jpg
qr_codes/images/325

qr_codes/images/2870381761.jpg
qr_codes/images/3362030142.jpg
qr_codes/images/2236526027.jpg
qr_codes/images/5137889651.jpg
qr_codes/images/4859558749.jpg
qr_codes/images/4956470281.jpg
qr_codes/images/3430305612.jpg
qr_codes/images/4219788005.jpg
qr_codes/images/2644241344.jpg
qr_codes/images/4123610847.jpg
qr_codes/images/2237232238.jpg
qr_codes/images/4195660045.jpg
qr_codes/images/4992344523.jpg
qr_codes/images/4848390238.jpg
qr_codes/images/3950055651.jpg
qr_codes/images/2708471549.jpg
qr_codes/images/2943752525.jpg
qr_codes/images/3482714875.jpg
qr_codes/images/1840578464.jpg
qr_codes/images/3346833848.jpg
qr_codes/images/2124747258.jpg
qr_codes/images/4952260665.jpg
qr_codes/images/4387464054.jpg
qr_codes/images/4884107014.jpg
qr_codes/images/4864169133.jpg
qr_codes/images/4440799716.jpg
qr_codes/images/4064425091.jpg
qr_codes/images/2985146218.jpg
qr_codes/images/4933142246.jpg
qr_codes/images/2237238420.jpg
qr_codes/images/2194489491.jpg
qr_codes/images/3897888195.jpg
qr_codes

qr_codes/images/3875393950.jpg
qr_codes/images/2595337516.jpg
qr_codes/images/5053297805.jpg
qr_codes/images/3360046942.jpg
qr_codes/images/4437534340.jpg
qr_codes/images/4059157596.jpg
qr_codes/images/5108412472.jpg
qr_codes/images/5164857092.jpg
qr_codes/images/4353977850.jpg
qr_codes/images/4391955186.jpg
qr_codes/images/4886984392.jpg
qr_codes/images/3891459413.jpg
qr_codes/images/4952852960.jpg
qr_codes/images/2236468419.jpg
qr_codes/images/5100495935.jpg
qr_codes/images/2595337520.jpg
qr_codes/images/2869227993.jpg
qr_codes/images/5085559548.jpg
qr_codes/images/3352436654.jpg
qr_codes/images/5055321464.jpg
qr_codes/images/0525173503.jpg
qr_codes/images/5149194708.jpg
qr_codes/images/2954565832.jpg
qr_codes/images/4864066718.jpg
qr_codes/images/4370993698.jpg
qr_codes/images/5241281672.jpg
qr_codes/images/4880150149.jpg
qr_codes/images/1439079783.jpg
qr_codes/images/5030691999.jpg
qr_codes/images/5262310913.jpg
qr_codes/images/5011786146.jpg
qr_codes/images/4604913130.jpg
qr_codes

In [3]:
from ppn_keras import metrics
from ppn_keras.networks.subparts_ssd_ppn_resnet50 import SubParts_SSD_PPN_ResNet50

network = SubParts_SSD_PPN_ResNet50(class_labels = ['qr_code'],
                                     subparts_class_labels = ['fip'],
                                     input_shape = (480,480,3))

import keras
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'resnet50': keras.applications.resnet50,
                        'relu6': keras.layers.ReLU(6.),
                        'ppn_loss': metrics.get_ppn_loss()}):
    network.model = keras.models.load_model(model_file)
    
network.model.summary()

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 480, 3)  0                                            
__________________________________________________________________________________________________
preprocess (Lambda)             (None, 480, 480, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 486, 3)  0           preprocess[0][0]                 
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 240, 240, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (

In [4]:
import pandas as pd
from ppn_keras.batch_generator import BatchGenerator

test_qr_codes = pd.read_csv('qr_codes_test.csv', dtype={'image_id': str, 'object_id': str})

batch_generator_test = BatchGenerator(network)
batch_generator_test.add_data(dataset = test_qr_codes, images_dir='images')

test_generator, test_size = batch_generator_test.get_generator(batch_size = 1, shuffle = False, augmentation = False, encode_output = False)
test_X, test_y = [], []

for _ in range(test_size):
  batch_X, batch_y = next(test_generator)
  test_X.append(batch_X[0])
  test_y.append(batch_y[0])
  
test_X = np.array(test_X)

Preprocessing Dataset: 100%|██████████| 158/158 [00:04<00:00, 32.03it/s]


In [5]:
y_pred = network.model.predict(test_X, verbose = 1, batch_size = 32)[1]

from ppn_keras.encode_decode_output import output_decoder
y_pred = output_decoder(y_pred, network, nms_threshold = 0.3)

print(metrics.AP(test_y, y_pred))
print(metrics.recall(test_y, y_pred))
print(metrics.false_positives(test_y, y_pred), '/', len(test_qr_codes))

100/100 [==============================] - 12s 116ms/step
0.7692193145169527
0.810126582278481
66 / 158
